# Chapitre 13 : Graphes pondérés

***Structure, implémentation et utilisation de graphes pondérés***

## Partie A - Représentation d'un graphe pondéré en mémoire

Un **graphe pondéré** est un graphe dont les arcs ou les arêtes portent une valeur numérique, appelée **poids**.

<img src='https://ntoulzac.github.io/Cours-NSI-Terminale/graphes_pond/images/graphe2_pond.png'>

**(1)** Ecrire la matrice d'adjacence associée au graphe ci-dessus. Au lieu de repérer par `1` la présence d'une arête entre deux sommets, on écrit dans la matrice le poids de l'arête.

In [1]:
mat_adj = [[0, 8, 0, 5, 0, 0, 2, 0, 0],
           [8, 0, 0, 0, 2, 0, 4, 0, 0],
           [0, 0, 0, 0, 0, 1, 0, 0, 3],
           [5, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 2, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 1, 0, 0, 0, 0, 1, 0],
           [2, 4, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 1, 0, 0, 0],
           [0, 0, 3, 0, 0, 0, 0, 0, 0]]

**(2)** Ecrire le dictionnaire d'adjacence associé au graphe ci-dessus. Au lieu de lister simplement les voisins de chaque sommet, on liste des couples de la forme `(voisin, poids)`.

In [2]:
dico_adj = {0 : [(1, 8), (3, 5), (6, 2)],
            1 : [(0, 8), (4, 2), (6, 4)],
            2 : [(5, 1), (8, 3)],
            3 : [(0, 5)],
            4 : [(1, 2)],
            5 : [(2, 1), (7, 1)],
            6 : [(0, 2), (1, 4)],
            7 : [(5, 1)],
            8 : [(2, 3)]}

**(3)** Modifier la définition des deux fonctions suivantes pour les adapter au cas d'un graphe pondéré.

In [3]:
def matrice_vers_dictionnaire_pond(mat):
    """
    Détermine le dictionnaire d'adjacence d'un graphe pondéré à partir de sa matrice d'adjacence.
    
    >>> matrice_vers_dictionnaire_pond(mat_adj) == dico_adj
    True
    """
    taille = len(mat)
    dico = {}
    for i in range(taille):
        dico[i] = []
        for j in range(taille):
            if mat[i][j] > 0:
                dico[i].append((j, mat[i][j]))
    return dico

In [4]:
import doctest as dt
dt.run_docstring_examples(matrice_vers_dictionnaire_pond, globals())

In [5]:
def dictionnaire_vers_matrice_pond(dico):
    """
    Détermine la matrice d'adjacence d'un graphe pondéré à partir de son dictionnaire d'adjacence.
    
    >>> dictionnaire_vers_matrice_pond(dico_adj) == mat_adj
    True
    """
    taille = len(dico)
    mat = [[0 for _ in range(taille)] for _ in range(taille)]
    for i in dico:
        for (j, poids) in dico[i]:
            mat[i][j] = poids
    return mat

In [6]:
import doctest as dt
dt.run_docstring_examples(dictionnaire_vers_matrice_pond, globals())

## Partie B - Arbre couvrant de poids minimal

Un **arbre couvrant**, pour un graphe connexe, est un arbre dont les branches sont des arêtes du graphe et qui connecte tous les sommets du graphe. Dans le cas d'un graphe non connexe, on parle de **forêt couvrante**.

<table class='sobre'>
    <tr>
        <td><img src='https://ntoulzac.github.io/Cours-NSI-Terminale/graphes_pond/images/arbre_couvrant_graphe.png'><br><i>Exemple de graphe</i></td>
        <td><img src='https://ntoulzac.github.io/Cours-NSI-Terminale/graphes_pond/images/arbre_couvrant_graphe1.png'><br><i>Premier exemple d'arbre couvrant</i></td>
        <td><img src='https://ntoulzac.github.io/Cours-NSI-Terminale/graphes_pond/images/arbre_couvrant_graphe2.png'><br><i>Deuxième exemple d'arbre couvrant</i></td>
    </tr>
</table>

Dans le cas d'un graphe pondéré, on peut se demander quel arbre couvrant est de poids minimal.

Le graphe ci-dessous montre comment sept petits villages sont reliés entre eux par un réseau de chemins. La longueur (exprimée en km) de chaque chemin est indiquée. Les responsables locaux souhaitent goudronner certains chemins, de sorte que tous les villages soient reliés par une route, tout en réalisant un nombre minimal de kilomètres de route.

<div>
    <img src='https://ntoulzac.github.io/Cours-NSI-Terminale/graphes_pond/images/glouton_exemple.png'>
</div>

**(4)** Déterminer une solution pour ce problème et expliquer la méthode utilisée.

<div class='rq'>Il existe plusieurs algorithmes gloutons permettant d'obtenir un arbre couvrant de poids minimal pour un graphe donné. Nous allons découvrir en particulier les algorithmes de Prim et de Kruskal.</div>

### Algorithme de Prim

<div class='rq3'>
<p><b>Algorithme de Prim pour la recherche d'un arbre couvrant de poids minimal dans un graphe</b></p>
<p><b>Début</b></p>
<p STYLE="padding:0 0 0 40px;"><code>arbre</code> ← ensemble vide</p>    
<p STYLE="padding:0 0 0 40px;"><code>s0</code> ← sommet du graphe choisi arbitrairement</p>
    <p STYLE="padding:0 0 0 40px;"><code>S</code> ← ensemble contenant un seul élément : le sommet <code>s0</code></p>
<p STYLE="padding:0 0 0 40px;">Tant que <code>S</code> ne contient pas tous les sommets du graphe faire</p>
    <p STYLE="padding:0 0 0 80px;"><code>a</code> ← plus courte arête parmi celles reliant un sommet <code>s1</code> dans <code>S</code> à un sommet <code>s2</code> hors de <code>S</code></p>
<p STYLE="padding:0 0 0 80px;">Insérer l'élément <code>s2</code> dans l'ensemble <code>S</code></p>
<p STYLE="padding:0 0 0 80px;">Insérer l'élément <code>a</code> dans l'ensemble <code>arbre</code></p>
<p STYLE="padding:0 0 0 40px;">Fin Tant que</p>
    <p STYLE="padding:0 0 0 40px;">Retourner <code>arbre</code></p>
<p><b>Fin</b></p>
</div>

**(5)** Exécuter à la main l'algorithme de Prim dans le cas du problème des sept villages.

<div>
    <img src='https://ntoulzac.github.io/Cours-NSI-Terminale/graphes_pond/images/glouton_prim_exemple.gif'>
    <center><i>Illustration de l'algorithme de Prim</i></center>
</div>

<div>
    <table>
        <tr>
            <th><center>Etape</center></th>
            <th><center>Sommets<br>reliés <code>S</code></center></th>
            <th><center>Plus petite<br>arête <code>a</code></center></th>
            <th><center>Nouveau sommet<br>relié <code>s2</code></center></th>
        </tr>
        <tr>
            <td><center>1</center></td>
            <td><center>A</center></td>
            <td><center>A-D (5)</center></td>
            <td><center>D</center></td>
        </tr>
        <tr>
            <td><center>2</center></td>
            <td><center>AD</center></td>
            <td><center>D-F (6)</center></td>
            <td><center>F</center></td>
        </tr>
        <tr>
            <td><center>3</center></td>
            <td><center>ADF</center></td>
            <td><center>A-B (7)</center></td>
            <td><center>B</center></td>
        </tr>
        <tr>
            <td><center>4</center></td>
            <td><center>ABDF</center></td>
            <td><center>B-E (7)</center></td>
            <td><center>E</center></td>
        </tr>
        <tr>
            <td><center>5</center></td>
            <td><center>ABDEF</center></td>
            <td><center>E-C (5)</center></td>
            <td><center>C</center></td>
        </tr>
        <tr>
            <td><center>6</center></td>
            <td><center>ABCDEF</center></td>
            <td><center>E-G (9)</center></td>
            <td><center>G</center></td>
        </tr>
    </table>
</div>

### Algorithme de Kruskal

<div class='rq3'>
<p><b>Algorithme de Kruskal pour la recherche d'un arbre couvrant de poids minimal dans un graphe</b></p>
<p><b>Début</b></p>
<p STYLE="padding:0 0 0 40px;"><code>arbre</code> ← ensemble vide</p>    
<p STYLE="padding:0 0 0 40px;"><code>A</code> ← tableau contenant toutes les arêtes du graphe classées par ordre croissant de poids</p>
    <p STYLE="padding:0 0 0 40px;">Pour <code>k</code> allant de 0 à longueur(<code>A</code>)-1 faire<p>
    <p STYLE="padding:0 0 0 80px;">Si les deux extrémités <code>s1</code> et <code>s2</code> de l'arête <code>A[k]</code> ne sont pas déjà reliées entre elles par des éléments de <code>arbre</code> alors</p>
<p STYLE="padding:0 0 0 120px;">Insérer l'élément <code>A[k]</code> dans l'ensemble <code>arbre</code></p>
        <p STYLE="padding:0 0 0 80px;">Fin Si</p>
<p STYLE="padding:0 0 0 80px;"><code>k</code> ← <code>k</code> + 1</p>
<p STYLE="padding:0 0 0 40px;">Fin Pour</p>
    <p STYLE="padding:0 0 0 40px;">Retourner <code>arbre</code></p>
<p><b>Fin</b></p>
</div>

**(6)** Exécuter à la main l'algorithme de Kruskal dans le cas du problème des sept villages.

<div>
    <img src='https://ntoulzac.github.io/Cours-NSI-Terminale/graphes_pond/images/glouton_kruskal_exemple.gif'>
</div>

<div>
    <table>
        <tr>
            <th><center>Etape<br><code>k</code></center></th>
            <th><center>Arête<br><code>A[k]</code></center></th>
            <th><center>Sommets<br>reliés ?</center></th>
            <th><center>Nouveaux<br>sommets reliés</center></th>
        </tr>
        <tr>
            <td><center>0</center></td>
            <td><center>C-E (5)</center></td>
            <td><center><i>non</i></center></td>
            <td><center>CE</center></td>
        </tr>
        <tr>
            <td><center>1</center></td>
            <td><center>A-D (5)</center></td>
            <td><center><i>non</i></center></td>
            <td><center>AD / CE</center></td>
        </tr>
        <tr>
            <td><center>2</center></td>
            <td><center>D-F (6)</center></td>
            <td><center><i>non</i></center></td>
            <td><center>ADF / CE</center></td>
        </tr>
        <tr>
            <td><center>3</center></td>
            <td><center>B-E (7)</center></td>
            <td><center><i>non</i></center></td>
            <td><center>ADF / BCE</center></td>
        </tr>
        <tr>
            <td><center>4</center></td>
            <td><center>A-B (7)</center></td>
            <td><center><i>non</i></center></td>
            <td><center>ABCDEF</center></td>
        </tr>
        <tr>
            <td><center>5</center></td>
            <td><center>E-F (8)</center></td>
            <td><center><i>oui</i></center></td>
            <td><center>ABCDEF</center></td>
        </tr>
        <tr>
            <td><center>6</center></td>
            <td><center>B-C (8)</center></td>
            <td><center><i>oui</i></center></td>
            <td><center>ABCDEF</center></td>
        </tr>
        <tr>
            <td><center>7</center></td>
            <td><center>E-G (9)</center></td>
            <td><center><i>non</i></center></td>
            <td><center>ABCDEFG</center></td>
        </tr>
        <tr>
            <td><center>8</center></td>
            <td><center>B-D (9)</center></td>
            <td><center><i>oui</i></center></td>
            <td><center>ABCDEFG</center></td>
        </tr>
        <tr>
            <td><center>9</center></td>
            <td><center>F-G (11)</center></td>
            <td><center><i>oui</i></center></td>
            <td><center>ABCDEFG</center></td>
        </tr>
    </table>
</div>

## Partie C - Plus court chemin entre deux sommets

**(7)** Dans le graphe pondéré ci-dessous, déterminer quelle est la longueur de la plus courte chaîne entre le sommet `0` et le sommet `9`.

<img src='https://ntoulzac.github.io/Cours-NSI-Terminale/graphes_pond/images/graphe3_pond.png'>

<div class='rq'>La plus courte chaîne entre le sommet <code>0</code> et le sommet <code>9</code> passe successivement par les sommets <code>0</code>, <code>1</code>, <code>3</code>, <code>4</code>,<code>7</code> et <code>9</code>.</div>

L'algorithme de Dijkstra permet de déterminer, dans un graphe orienté (ou non orienté), le plus court chemin (ou la plus courte chaîne) permettant d'atteindre n'importe quel sommet à partir d'un sommet `depart`.

<div class='rq3'>
<p><b>Algorithme de Dijkstra pour la recherche du plus court chemin permettant d'atteindre n'importe quel sommet à partir d'un sommet <code>depart</code></b></p>
<p><b>Début</b></p>
<p STYLE="padding:0 0 0 40px;"><code>dist</code> ← dictionnaire dont les clés sont les sommets du graphe et la valeur associée à chaque clé est $+\infty$</p> 
    <p STYLE="padding:0 0 0 40px;"><code>pred</code> ← dictionnaire dont les clés sont les sommets du graphe et la valeur associée à chaque clé est <code>None</code></p> 
<p STYLE="padding:0 0 0 40px;"><code>dist[depart]</code> ← 0</p>    
<p STYLE="padding:0 0 0 40px;">Tant que tous les sommets du <code>graphe</code> ne sont pas marqués faire</p>
    <p STYLE="padding:0 0 0 80px;"><code>sommet</code> ← sommet non marqué du <code>graphe</code> tel que <code>dist[sommet]</code> soit minimal</p>
<p STYLE="padding:0 0 0 80px;">Marquer le <code>sommet</code></p>
    <p STYLE="padding:0 0 0 80px;">Pour chaque <code>voisin</code> non marqué de <code>sommet</code> faire</p>
  <p STYLE="padding:0 0 0 120px;">Si <code>dist[sommet]</code> + <code>poids_arete(sommet, voisin)</code> &lt; <code>dist[voisin]</code> alors</p> 
      <p STYLE="padding:0 0 0 160px;"><code>dist[voisin]</code> ← <code>dist[sommet]</code> + <code>poids_arete(sommet, voisin)</code></p> 
          <p STYLE="padding:0 0 0 160px;"><code>pred[voisin]</code> ← <code>sommet</code></p> 
      <p STYLE="padding:0 0 0 120px;">Fin Si</p> 
        <p STYLE="padding:0 0 0 80px;">Fin Pour</p>
<p STYLE="padding:0 0 0 40px;">Fin Tant que</p>
    <p STYLE="padding:0 0 0 40px;">Retourner <code>dist</code>, <code>pred</code></p>
<p><b>Fin</b></p>
</div>

**(8)** Appliquer à la main l'algorithme de Dijkstra pour retrouver la plus courte chaîne entre le sommet `0` et le sommet `9`.

<img src='https://ntoulzac.github.io/Cours-NSI-Terminale/graphes_pond/images/graphe3_pond.png'>

<table>
    <tr>
        <th><code>sommet</code></th>
        <th><code>0</code></th>
        <th><code>1</code></th>
        <th><code>2</code></th>
        <th><code>3</code></th>
        <th><code>4</code></th>
        <th><code>5</code></th>
        <th><code>6</code></th>
        <th><code>7</code></th>
        <th><code>8</code></th>
        <th><code>9</code></th>
    </tr>
    <tr>
        <td></td>
        <td><b>0</b></td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
    </tr>
    <tr>
        <td><code>0</code></td>
        <td></td>
        <td>3 (<code>0</code>)</td>
        <td><b>2 (<code>0</code>)</b></td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
    </tr>
    <tr>
        <td><code>2</code></td>
        <td></td>
        <td><b>3 (<code>0</code>)</b></td>
        <td></td>
        <td>6 (<code>2</code>)</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
    </tr>
    <tr>
        <td><code>1</code></td>
        <td></td>
        <td></td>
        <td></td>
        <td><b>4 (<code>1</code>)</b></td>
        <td>$$+\infty$$</td>
        <td>8 (<code>1</code>)</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
    </tr>
    <tr>
        <td><code>3</code></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td><b>6 (<code>3</code>)</b></td>
        <td>8 (<code>1</code>)</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
    </tr>
    <tr>
        <td><code>4</code></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td><b>7 (<code>4</code>)</b></td>
        <td>10 (<code>4</code>)</td>
        <td>8 (<code>4</code>)</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
    </tr>
    <tr>
        <td><code>5</code></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td>10 (<code>4</code>)</td>
        <td><b>8 (<code>4</code>)</b></td>
        <td>10 (<code>5</code>)</td>
        <td>$$+\infty$$</td>
    </tr>
    <tr>
        <td><code>7</code></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td><b>10 (<code>4</code>)</b></td>
        <td></td>
        <td>10 (<code>5</code>)</td>
        <td>10 (<code>7</code>)</td>
    </tr>
    <tr>
        <td><code>6</code></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td><b>10 (<code>5</code>)</b></td>
        <td>10 (<code>7</code>)</td>
    </tr>
    <tr>
        <td><code>8</code></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td><b>10 (<code>7</code>)</b></td>
    </tr>
</table>

<div class='rq'>Le dictionnaire <code>dist</code> associe à chaque sommet sa distance par rapport au sommet de départ (ici le sommet <code>0</code>).</div>

<table>
    <tr>
        <th><code>dist[0]</code></th>
        <th><code>dist[1]</code></th>
        <th><code>dist[2]</code></th>
        <th><code>dist[3]</code></th>
        <th><code>dist[4]</code></th>
        <th><code>dist[5]</code></th>
        <th><code>dist[6]</code></th>
        <th><code>dist[7]</code></th>
        <th><code>dist[8]</code></th>
        <th><code>dist[9]</code></th>
    </tr>
    <tr>
        <td>0</td>
        <td>3</td>
        <td>2</td>
        <td>4</td>
        <td>6</td>
        <td>7</td>
        <td>10</td>
        <td>8</td>
        <td>10</td>
        <td>10</td>
    </tr>
</table>

<div class='rq'>Le dictionnaire <code>pred</code> associe à chaque sommet le voisin par lequel on y arrive si on emprunte la plus courte chaîne à partir du départ (ici le sommet <code>0</code>).</div>

<table>
    <tr>
        <th><code>pred[0]</code></th>
        <th><code>pred[1]</code></th>
        <th><code>pred[2]</code></th>
        <th><code>pred[3]</code></th>
        <th><code>pred[4]</code></th>
        <th><code>pred[5]</code></th>
        <th><code>pred[6]</code></th>
        <th><code>pred[7]</code></th>
        <th><code>pred[8]</code></th>
        <th><code>pred[9]</code></th>
    </tr>
    <tr>
        <td><code>None</code></td>
        <td><code>0</code></td>
        <td><code>0</code></td>
        <td><code>1</code></td>
        <td><code>3</code></td>
        <td><code>4</code></td>
        <td><code>4</code></td>
        <td><code>4</code></td>
        <td><code>5</code></td>
        <td><code>7</code></td>
    </tr>
</table>

**(9)** Compléter l'implémentation de l'algorithme de Dijkstra donnée ci-dessous.

In [7]:
def dijkstra(dico, depart):
    """
    Applique l'algorithme de Dijkstra, à partir d'un sommet depart, dans un graphe représenté par son
    dictionnaire d'adjacence.
    - Entrées : dico (dictionnaire d'adjacence du graphe), depart (sommet du graphe)
    - Sortie : dist, predecesseur (dictionnaires, dist associe à chaque sommet sa dist. au sommet de depart,
               predecesseur associe à chaque sommet le sommet d'où venir pour minimiser la dist. au départ)
    """
    def indice_sommet_a_traiter():
        """
        Recherche dans le tableau sommets_non_marques l'indice du sommet qui est le plus proche de depart.
        - Sortie : (entier, indice d'un élément du tableau sommets_non_marques)
        """
        indice = 0
        for i in range(1, len(sommets_non_marques)):
            if dist[sommets_non_marques[i]] < dist[sommets_non_marques[indice]]:
                indice = i
        return indice
    
    dist = {sommet : float('inf') for sommet in dico}
    pred = {sommet : None for sommet in dico}
    dist[depart] = 0
    sommets_non_marques = [sommet for sommet in dico]
    while len(sommets_non_marques) > 0:
        indice = indice_sommet_a_traiter()
        sommet = sommets_non_marques[indice]
        del sommets_non_marques[indice] # On "marque" le sommet en le supprimant du tableau sommets_non_marques
        for (voisin, poids) in dico[sommet]:
            if voisin in sommets_non_marques and dist[sommet] + poids < dist[voisin]:
                dist[voisin] = dist[sommet] + poids
                pred[voisin] = sommet
    return dist, pred

In [8]:
dico_adj = {0 : [(1, 3), (2, 2)],
            1 : [(0, 3), (2, 2), (3, 1), (5, 5)],
            2 : [(0, 2), (1, 2), (3, 4)],
            3 : [(1, 1), (2, 4), (4, 2)],
            4 : [(3, 2), (5, 1), (6, 4), (7, 2)],
            5 : [(1, 5), (4, 1), (7, 1), (8, 3)],
            6 : [(4, 4), (7, 5)],
            7 : [(4, 2), (5, 1), (6, 5), (9, 2)],
            8 : [(5, 3), (9, 3)],
            9 : [(7, 2), (8, 3)]}

distance, precedent = dijkstra(dico_adj, 0)

**(10)** Afficher, pour chaque sommet, sa distance au sommet `0`.

In [9]:
for sommet in range(10):
    print(f"Sommet {sommet} : distance {distance[sommet]} à partir du sommet 0")

Sommet 0 : distance 0 à partir du sommet 0
Sommet 1 : distance 3 à partir du sommet 0
Sommet 2 : distance 2 à partir du sommet 0
Sommet 3 : distance 4 à partir du sommet 0
Sommet 4 : distance 6 à partir du sommet 0
Sommet 5 : distance 7 à partir du sommet 0
Sommet 6 : distance 10 à partir du sommet 0
Sommet 7 : distance 8 à partir du sommet 0
Sommet 8 : distance 10 à partir du sommet 0
Sommet 9 : distance 10 à partir du sommet 0


**(11)** Afficher la plus courte chaîne permettant d'atteindre chaque sommet à partir du sommet `0`.

In [10]:
for sommet in range(10):
    s = sommet
    L = []
    while s is not None:
        L.append(s)
        s = precedent[s]
    if L[-1] == 0:
        L.reverse()
    else:
        L = []
    print(f"Plus courte chaîne permettant d'atteindre le sommet {sommet} à partir du sommet 0 : {L}")

Plus courte chaîne permettant d'atteindre le sommet 0 à partir du sommet 0 : [0]
Plus courte chaîne permettant d'atteindre le sommet 1 à partir du sommet 0 : [0, 1]
Plus courte chaîne permettant d'atteindre le sommet 2 à partir du sommet 0 : [0, 2]
Plus courte chaîne permettant d'atteindre le sommet 3 à partir du sommet 0 : [0, 1, 3]
Plus courte chaîne permettant d'atteindre le sommet 4 à partir du sommet 0 : [0, 1, 3, 4]
Plus courte chaîne permettant d'atteindre le sommet 5 à partir du sommet 0 : [0, 1, 3, 4, 5]
Plus courte chaîne permettant d'atteindre le sommet 6 à partir du sommet 0 : [0, 1, 3, 4, 6]
Plus courte chaîne permettant d'atteindre le sommet 7 à partir du sommet 0 : [0, 1, 3, 4, 7]
Plus courte chaîne permettant d'atteindre le sommet 8 à partir du sommet 0 : [0, 1, 3, 4, 5, 8]
Plus courte chaîne permettant d'atteindre le sommet 9 à partir du sommet 0 : [0, 1, 3, 4, 7, 9]


## Exercices

In [11]:
from structures_relationnelles import GrapheOriente

Problème d'importation de graphviz


### Exercice 1

La carte ci-dessous représente 21 villes reliées entre elles par des segments dont la longueur est exprimée en dizaines de kilomètres.

Par exemple, la valeur 34 sur le segment reliant Paris et Londres signifie que les deux villes sont distantes de 340 km.

<div>
    <img src='https://ntoulzac.github.io/Cours-NSI-Terminale/graphes_pond/images/glouton_france_dep1.png'>
</div>

Déterminer quelles liaisons conserver et quelles liaisons supprimer pour minimiser la longueur totale des liaisons, sachant que toutes les villes doivent rester reliées entre elles.

<div class='rq'>
    <table class='sobre'>
        <tr>
            <td><img src='https://ntoulzac.github.io/Cours-NSI-Terminale/graphes_pond/images/glouton_france_dep2.png'>
            <center><br><i>Graphe</i></center></td>
            <td><img src='https://ntoulzac.github.io/Cours-NSI-Terminale/graphes_pond/images/glouton_france_arr.png'>
    <center><br><i>Arbre couvrant de poids minimal</i></center></td>
        </tr>
    </table>
<br>
   Un arbre couvrant de poids minimal peut être obtenu avec l'algorithme de Prim ou l'algorithme de Kruskal. La longueur totale des liaisons conservées est de 4230 km.
</div>

### Exercice 2

**(1)** Appliquer à la main l'algorithme de Dijkstra pour trouver la plus courte chaîne entre le sommet `3` et le sommet `4`.

<img src='https://ntoulzac.github.io/Cours-NSI-Terminale/graphes_pond/images/graphe2_pond.png'>

<table>
    <tr>
        <th><code>sommet</code></th>
        <th><code>0</code></th>
        <th><code>1</code></th>
        <th><code>2</code></th>
        <th><code>3</code></th>
        <th><code>4</code></th>
        <th><code>5</code></th>
        <th><code>6</code></th>
        <th><code>7</code></th>
        <th><code>8</code></th>
    </tr>
    <tr>
        <td></td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td><b>0</b></td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
    </tr>
    <tr>
        <td><code>3</code></td>
        <td><b>5 (<code>3</code>)</b></td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td></td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
    </tr>
    <tr>
        <td><code>0</code></td>
        <td></td>
        <td>13 (<code>0</code>)</td>
        <td>$$+\infty$$</td>
        <td></td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td><b>7 (<code>0</code>)</b></td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
    </tr>
    <tr>
        <td><code>6</code></td>
        <td></td>
        <td><b>11 (<code>6</code>)</b></td>
        <td>$$+\infty$$</td>
        <td></td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
        <td></td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
    </tr>
    <tr>
        <td><code>1</code></td>
        <td></td>
        <td></td>
        <td>$$+\infty$$</td>
        <td></td>
        <td><b>13 (<code>1</code>)</b></td>
        <td>$$+\infty$$</td>
        <td></td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
    </tr>
    <tr>
        <td><code>4</code></td>
        <td></td>
        <td></td>
        <td><b>$$+\infty$$</b></td>
        <td></td>
        <td></td>
        <td>$$+\infty$$</td>
        <td></td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
    </tr>
    <tr>
        <td><code>2</code></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td><b>$$+\infty$$</b></td>
        <td></td>
        <td>$$+\infty$$</td>
        <td>$$+\infty$$</td>
    </tr>
    <tr>
        <td><code>5</code></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td><b>$$+\infty$$</b></td>
        <td>$$+\infty$$</td>
    </tr>
    <tr>
        <td><code>7</code></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td><b>$$+\infty$$</b></td>
    </tr>
</table>

**(2)** Afficher, pour chaque sommet, la chaîne la plus courte (lorsqu'elle existe) permettant d'atteindre ce sommet à partir du sommet `0`, ainsi que la longueur de cette chaîne.

In [12]:
dico_adj = {0 : [(1, 8), (3, 5), (6, 2)],
            1 : [(0, 8), (4, 2), (6, 4)],
            2 : [(5, 1), (8, 3)],
            3 : [(0, 5)],
            4 : [(1, 2)],
            5 : [(2, 1), (7, 1)],
            6 : [(0, 2), (1, 4)],
            7 : [(5, 1)],
            8 : [(2, 3)]}

distance, precedent = dijkstra(dico_adj, 3)

for sommet in range(9):
    if distance[sommet] < float('inf'):
        print(f"Sommet {sommet} :")
        s = sommet
        L = []
        while s is not None:
            L.append(s)
            s = precedent[s]
        if L[-1] == 3:
            L.reverse()
        else:
            L = []
        print(f"- Plus courte chaîne : {L}")
        print(f"- Longueur : {distance[sommet]}")

Sommet 0 :
- Plus courte chaîne : [3, 0]
- Longueur : 5
Sommet 1 :
- Plus courte chaîne : [3, 0, 6, 1]
- Longueur : 11
Sommet 3 :
- Plus courte chaîne : [3]
- Longueur : 0
Sommet 4 :
- Plus courte chaîne : [3, 0, 6, 1, 4]
- Longueur : 13
Sommet 6 :
- Plus courte chaîne : [3, 0, 6]
- Longueur : 7
